In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO

## Get the text from the Wiki page and parse it with lxml

In [12]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

## Extract the table content 

In [3]:
table_content = soup.find('table').text
print(table_content)



Postcode
Borough
Neighbourhood


M1A
Not assigned
Not assigned


M2A
Not assigned
Not assigned


M3A
North York
Parkwoods


M4A
North York
Victoria Village


M5A
Downtown Toronto
Harbourfront


M5A
Downtown Toronto
Regent Park


M6A
North York
Lawrence Heights


M6A
North York
Lawrence Manor


M7A
Queen's Park
Not assigned


M8A
Not assigned
Not assigned


M9A
Etobicoke
Islington Avenue


M1B
Scarborough
Rouge


M1B
Scarborough
Malvern


M2B
Not assigned
Not assigned


M3B
North York
Don Mills North


M4B
East York
Woodbine Gardens


M4B
East York
Parkview Hill


M5B
Downtown Toronto
Ryerson


M5B
Downtown Toronto
Garden District


M6B
North York
Glencairn


M7B
Not assigned
Not assigned


M8B
Not assigned
Not assigned


M9B
Etobicoke
Cloverdale


M9B
Etobicoke
Islington


M9B
Etobicoke
Martin Grove


M9B
Etobicoke
Princess Gardens


M9B
Etobicoke
West Deane Park


M1C
Scarborough
Highland Creek


M1C
Scarborough
Rouge Hill


M1C
Scarborough
Port Union


M2C
Not assigned
Not assigned

## Split the table content by newlines in order to obtain a list of words. Then, remove empty elements from the list.

In [4]:
#split
table_list = table_content.split('\n')

#remove empty elements
table_list = list(filter(None, table_list))

#print first 20 elements:
print(table_list[:20])

['Postcode', 'Borough', 'Neighbourhood', 'M1A', 'Not assigned', 'Not assigned', 'M2A', 'Not assigned', 'Not assigned', 'M3A', 'North York', 'Parkwoods', 'M4A', 'North York', 'Victoria Village', 'M5A', 'Downtown Toronto', 'Harbourfront', 'M5A', 'Downtown Toronto']


## Seperate Labels and Records

In [5]:
labels = ['PostalCode', 'Borough', 'Neighborhood']
records = table_list[3:]

In [6]:
column1 = []
for i in range(0,len(records)):
        if i%3 == 0:
            column1.append(records[i])

column2 = []
for i in range(0,len(records)):
        if (i-1)%3 == 0:
            column2.append(records[i])

column3 = []
for i in range(0,len(records)):
        if (i-2)%3 == 0:
            column3.append(records[i])

records_arranged = list(zip(column1, column2, column3))
records_arranged

[('M1A', 'Not assigned', 'Not assigned'),
 ('M2A', 'Not assigned', 'Not assigned'),
 ('M3A', 'North York', 'Parkwoods'),
 ('M4A', 'North York', 'Victoria Village'),
 ('M5A', 'Downtown Toronto', 'Harbourfront'),
 ('M5A', 'Downtown Toronto', 'Regent Park'),
 ('M6A', 'North York', 'Lawrence Heights'),
 ('M6A', 'North York', 'Lawrence Manor'),
 ('M7A', "Queen's Park", 'Not assigned'),
 ('M8A', 'Not assigned', 'Not assigned'),
 ('M9A', 'Etobicoke', 'Islington Avenue'),
 ('M1B', 'Scarborough', 'Rouge'),
 ('M1B', 'Scarborough', 'Malvern'),
 ('M2B', 'Not assigned', 'Not assigned'),
 ('M3B', 'North York', 'Don Mills North'),
 ('M4B', 'East York', 'Woodbine Gardens'),
 ('M4B', 'East York', 'Parkview Hill'),
 ('M5B', 'Downtown Toronto', 'Ryerson'),
 ('M5B', 'Downtown Toronto', 'Garden District'),
 ('M6B', 'North York', 'Glencairn'),
 ('M7B', 'Not assigned', 'Not assigned'),
 ('M8B', 'Not assigned', 'Not assigned'),
 ('M9B', 'Etobicoke', 'Cloverdale'),
 ('M9B', 'Etobicoke', 'Islington'),
 ('M9B', 

## Create a DataFrame 

In [7]:
#create dataframe
df = pd.DataFrame.from_records(records_arranged, columns=labels)

#drop rows with Borough="Not assigned"
df = df[df["Borough"] != "Not assigned"]

df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## Group the DataFrame by PostalCode and thereby, join Neighborhoods with a comma and leave Boroughs as they are

In [8]:
#create custom function to join the Neighbourhood values with comma and space
customjoin = lambda a: ", ".join(a) 

#create a custom function to join the Borough values with comma, seperate them by comma and only return the first (since they are equal)
#I simply wanted to leave the Borough value unchanged. I'm sure there is a more elegant way, but this one works fine.
onereturn= lambda a: ",".join(a).rsplit(",")[0]

#group dataframe by PostalCode and join entries 
df_final = df.groupby('PostalCode').agg({'Borough': onereturn, 'Neighborhood': customjoin})

#check if the join works as expected:
df_final.head(4)

,Borough,Neighborhood
PostalCode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn


## Find rows with Neighborhood 'Not assigned' and replace it by the Borough entry

In [9]:
#check rows:
df_final[df_final['Neighborhood']=='Not assigned']
#only the case for PostalCode M7A. Replace its value by its Borough value:
df_final.loc['M7A']['Neighborhood'] = df_final.loc['M7A']['Borough']

#check again:
df_final[df_final['Neighborhood']=='Not assigned']
#no returns -> worked

,Borough,Neighborhood
PostalCode,,


## The columns Neighborhood and Borough are swapped, compared to the given example on Coursera. Therefore, I swap the columns.
## Finally, the index is reset and PostalCode becomes an ordinary column.

In [10]:
df_final=df_final.reindex(columns=['Borough', 'Neighborhood'])
df_final = df_final.reset_index()
df_final.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df_final.shape

(103, 3)